In [50]:
import findspark
findspark.init()
import pandas as pd;
import numpy as np;
from pyspark import SparkContext
from pyspark import SparkConf
import re

In [32]:
conf = SparkConf().setAppName('Read & parse text data file in pyspark')
sc = SparkContext.getOrCreate(conf=conf)

In [36]:
lines = sc.textFile('./assets/AMiner-Author.txt').zipWithIndex()

In [37]:
# Find the starting row of each data entry
pos = lines.filter(lambda x: "#index" in x[0]).map(lambda x: x[1]).collect()

In [38]:
zeroArray = np.zeros(lines.count(), dtype=int)

In [39]:
def exchangeElementAtIndexToOne(index):
    zeroArray[index] = 1;

In [40]:
for element in pos:
    exchangeElementAtIndexToOne(element)

In [41]:
# Calculate cumulative sum of starting data entry indexes
summedArray = np.cumsum(zeroArray)

In [42]:
broadcastedArray = sc.broadcast(summedArray);

In [43]:
def createBroarcastedTuple(x):
    return (broadcastedArray.value[x[1]], x[0])

In [44]:
papers = lines.map(lambda x: createBroarcastedTuple(x))

In [13]:
list_get = lambda l, x, d=None: d if not l[x:x+1] else l[x] # safe getter of list values

def reduceFeaturesToDict(featureA={}, featureB=''):
    try:
        splittedA = featureA;
        if type(splittedA) is str:
            splittedA = splittedA.split(" ", 1)
            if len(splittedA) > 0:
                splittedA = { list_get(splittedA, 0): list_get(splittedA, 1, '') }
            else:
                splittedA = {}

        splittedB = featureB;
        if type(featureB) is str:
            if len(featureB) > 0:
                splittedB = featureB.split(" ", 1)
                splittedB = { list_get(splittedB, 0): list_get(splittedB, 1, '') }
            else:
                splittedB = {}
        return { **splittedA, **splittedB }
    except:
        print("ERROR: ", featureA, featureB)
        raise

In [45]:
reducedPapers = papers.reduceByKey(lambda a, b: reduceFeaturesToDict(a, b))

In [ ]:
print(reducedPapers.sortByKey(ascending=False).first())

In [47]:
mappedPaperDicts = reducedPapers.map(lambda x: x[1]) # retrieve dicts from the tuples

In [ ]:
# print(mappedPaperDicts.sortByKey(ascending=False).take(100))

In [ ]:
# pdPapers = pd.DataFrame.from_records(mappedPaperDicts)

In [ ]:
mappedPaperDicts.saveAsTextFile('./assets/parsedData/authors')